## Handling Missing Values in Python During the Data Analysis Process
Missing values are a common occurrence in data analysis and must be handled effectively to ensure the accuracy of insights derived from the data. This document provides a comprehensive guide to handling missing values in Python, covering various techniques and their implementations.

### 1. Understanding Missing Values
### Causes of Missing Values:
- Data entry errors
- Non-responses in surveys
- Data corruption
- Merging datasets with unmatched keys

### Types of Missing Data:
- Missing Completely at Random (MCAR): No systematic pattern.
- Missing at Random (MAR): Systematic relationship with other variables.
- Missing Not at Random (MNAR): Related to the missing value itself.

#### Identifying Missing Values:

Before handling missing values, identify them in the dataset.

In [1]:
import pandas as pd

# Example dataset
data = {
    'Name': ['Sajjad', 'Noor', 'Sameer', None],
    'Age': [25, None, 30, 22],
    'Salary': [50000, 60000, None, 45000]
}
df = pd.DataFrame(data)

# Check for missing values
print(df.isnull())  # Boolean mask for missing values
print(df.isnull().sum())  # Count of missing values per column

    Name    Age  Salary
0  False  False   False
1  False   True   False
2  False  False    True
3   True  False   False
Name      1
Age       1
Salary    1
dtype: int64


### 2. Techniques to Handle Missing Values

#### 2.1 Dropping Missing Values
- When to use: If missing values are sparse and not critical.